In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Equi7Grid tile

An `Equi7Tile` is . Its implementation extends the definition of a `pytileproj`'s. [`RasterTile`](https://tuw-geo.github.io/pytileproj/latest/guides/tile.html#raster-tile). 

### Initialisation

The constructor of `RasterTile` expects the following arguments:

- `n_rows`: Number of rows or height in pixels of the raster
- `n_cols`: Number of columns or width in pixels of the raster
- `crs`: A spatial reference system represented by anything pyproj supports, e.g., EPSG code, PROJ4 string, WKT string, ...
- `geotrans` (optional): tuple containing the 6 GDAL affine geotransformation parameters
- `name` (optional): Name of the raster tile
- `px_origin` (optional): The world system origin of the pixels (cf. Properties section).

How the 6 geotransformation parameters relate to a georeferenced raster can be seen in the figure below. 


```{image} /guides/imgs/geotrans_params.png
   :align: center
   :width: 60%
```


In [ ]:
from pytileproj.tile import RasterTile

# initialisation of variables
n_rows = 50
n_cols = 50
epsg = 4326
geotrans = (
    5,
    0.2,
    0,
    50,
    0,
    -0.2,
)  # ul_x, x_pixel_size, x_rot, ul_y, y_rot, y_pixel_size
tilename = "Tile 1"
# additional, optional parameter is px_origin
RasterTile(n_rows=n_rows, n_cols=n_cols, crs=epsg, geotrans=geotrans, name=tilename)

This is not the only way to create a raster geometry. You can also initialise a raster geometry from a given extent, 

In [ ]:
extent = (5, 40, 15, 50)
RasterTile.from_extent(extent, epsg, 0.2, 0.2)

from a given projected geometry,

In [ ]:
import pyproj
from pytileproj.projgeom import ProjGeom
from shapely.geometry import Polygon

polygon = Polygon([(5, 40), (5, 50), (15, 50), (15, 40), (5, 40)])
proj_geom = ProjGeom(geom=polygon, crs=pyproj.CRS.from_epsg(4326))

RasterTile.from_geometry(proj_geom, 0.2, 0.2, name=tilename)

or from a JSON string:

In [ ]:
json_str = """
{
    "crs": 4326,
    "n_rows": 50,
    "n_cols": 50,
    "geotrans": [
        5.0,
        0.2,
        0.0,
        50.0,
        0.0,
        -0.2
    ],
    "px_origin": "ul",
    "name": "Tile 1"
}
"""
raster_tile = RasterTile.from_json(json_str)

All options above result in the same raster tile object.

### Properties

Now having a raster geometry instance ready, we can continue with inspecting the properties of this object. 
The shape of the geometry is defined by its width  

In [ ]:
raster_tile.width

and its height.

In [ ]:
raster_tile.height

Direct shape access is possible with (height, width)

In [ ]:
raster_tile.shape

The same can be done by using actual world system coordinates. The width is accessable through

In [ ]:
raster_tile.x_size

and the height through

In [ ]:
raster_tile.y_size

The sizes of each pixel can be easily retrieved via

In [ ]:
raster_tile.x_pixel_size

and 

In [ ]:
raster_tile.y_pixel_size

In the case of a rotated raster geometry there additional properties, which can be of interest, for instance the counter-clockwise orientation in radians with respect to the W-E direction/horizontal.

In [ ]:
raster_tile.ori

You can also check if your raster geometry is rotated or not with

In [ ]:
raster_tile.is_axis_parallel

A very important thing of the relationship between pixel and world system coordinates is the anchor point or pixel origin, i.e. to what point in the pixel the coordinate refers to. By default the GDAL definition is used, which states that the origin is in the upper-left corner of the upper-left pixel. But with a raster geometry, you also have the option to choose between all other corner points and the pixel center. 

The aformentioned properties and the different possibilities for the pixel origin are illustrated in the graphic below.


```{image} /guides/imgs/raster_geometry.png
   :align: center
   :width: 60%
```


If you are interested in the corner points of the raster geometry, several properties help you to access the respective coordinates. For example, the lower-left corner, i.o.w. the first pixel in the last row, has the following coordinates:

In [ ]:
raster_tile.ll_x, raster_tile.ll_y

If you want to know the full coordinate extent (x_min, y_min, x_max, y_max), you can call

In [ ]:
raster_tile.coord_extent

Please note that all these coordinates refer to the pixel origin, which has been chosen during class initialisation.
If you are interested in the full extent of the raster geometry (bold black line in the image before), you can use

In [ ]:
raster_tile.outer_boundary_extent

The corner points of the outer boundary are available via

In [ ]:
raster_tile.outer_boundary_corners

Note that they match `outer_boundary_extent` only for non-rotated raster geometries, but not for rotated ones. The full range of 1-D coordinates in a certain direction along the edges of a raster geometry can be retrieved with 

In [ ]:
raster_tile.x_coords

and 

In [ ]:
raster_tile.y_coords

These coordinates only refer to the first row or first column. They are representatives for the whole raster geometry in the non-rotated case, but not as soon as we are working with a rotated raster tile. If you are interested in every coordinate contained within the raster geometry you can use:

In [ ]:
raster_tile.xy_coords

The boundary of the raster tile is also availabe on a higher-level, e.g. as a WKT string, OGR or shapely geometry.

In [ ]:
raster_tile.boundary_wkt

Not all properties have been discussed here. Please take look at the documentation to explore the full range of offered functionality.

### Plotting

A very nice feature of a raster geometry is that you can plot it on a map. Several keywords can help you to beautify your plot. First, we can simply try to plot the raster geometry we have created before.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
raster_tile.plot()

Since the full extent of the projection is chosen by default, we can try limit the extent with:

In [ ]:
plt.figure(figsize=(12, 12))
raster_tile.plot(extent=(0, 35, 20, 55))

The plot function also allows to change the projection. We can try to define a new one, e.g. a Pseudo-Mercator projection, and use it for our new map.

In [ ]:
import pyproj

plt.figure(figsize=(12, 12))
raster_tile.plot(
    label_tile=True,
    facecolor="green",
    edgecolor="blue",
    edgewidth=5,
    alpha=0.7,
    proj=3857,
    extent=(0, 35, 20, 55),
)

### Topological Operations

Raster tiles provide the functionality to interact with each other or with other geometries. The first set of functions checks for certain topological constrains, e.g. WITHIN, INTERSECTS, TOUCHES or OVERLAPS. To try this out we can create additional raster tiles.  

In [ ]:
geotrans_within = (6, 0.2, 0, 49, 0, -0.2)
raster_tile_within = RasterTile(
    n_rows=25, n_cols=25, crs=epsg, geotrans=geotrans_within, name="WITHIN"
)

geotrans_outer = (1, 0.2, 0, 49, 0, -0.2)
raster_tile_outer = RasterTile(
    n_rows=15, n_cols=5, crs=epsg, geotrans=geotrans_outer, name="OUTER"
)

geotrans_overlap = (12, 0.2, 0, 45, 0, -0.2)
raster_tile_overlap = RasterTile(
    n_rows=15, n_cols=30, crs=epsg, geotrans=geotrans_overlap, name="OVERLAPS"
)

geotrans_touch = (10, 0.2, 0, 40, 0, -0.2)
raster_tile_touch = RasterTile(
    n_rows=35, n_cols=25, crs=epsg, geotrans=geotrans_touch, name="TOUCHES"
)

In [ ]:
plt.figure(figsize=(12, 12))
ax_1 = raster_tile.plot(extent=(0, 35, 20, 55))
raster_tile_within.plot(
    ax=ax_1,
    label_tile=True,
    facecolor="blue",
    edgecolor="blue",
    edgewidth=2,
    alpha=0.7,
    extent=(0, 35, 20, 55),
)
raster_tile_outer.plot(
    ax=ax_1,
    label_tile=True,
    facecolor="orange",
    edgecolor="orange",
    edgewidth=2,
    alpha=0.7,
    extent=(0, 35, 20, 55),
)
raster_tile_overlap.plot(
    ax=ax_1,
    label_tile=True,
    facecolor="green",
    edgecolor="green",
    edgewidth=2,
    alpha=0.7,
    extent=(0, 35, 20, 55),
)
raster_tile_touch.plot(
    ax=ax_1,
    label_tile=True,
    facecolor="purple",
    edgecolor="purple",
    edgewidth=2,
    alpha=0.7,
    extent=(0, 35, 20, 55),
)

In [ ]:
(
    raster_tile_within.within(raster_tile),
    raster_tile_within.intersects(raster_tile),
    raster_tile_within.touches(raster_tile),
    raster_tile_within.overlaps(raster_tile),
)

In [ ]:
(
    raster_tile_outer.within(raster_tile),
    raster_tile_outer.intersects(raster_tile),
    raster_tile_outer.touches(raster_tile),
    raster_tile_outer.overlaps(raster_tile),
)

In [ ]:
(
    raster_tile_overlap.within(raster_tile),
    raster_tile_overlap.intersects(raster_tile),
    raster_tile_overlap.touches(raster_tile),
    raster_tile_overlap.overlaps(raster_tile),
)

In [ ]:
(
    raster_tile_touch.within(raster_tile),
    raster_tile_touch.intersects(raster_tile),
    raster_tile_touch.touches(raster_tile),
    raster_tile_touch.overlaps(raster_tile),
)

### Coordinate Conversions

A raster tile also provides an interface for pixel to coordinate (`rc2xy`) and coordinate to pixel (`xy2rc`) conversions. 

In [ ]:
import numpy as np

rows, cols = np.meshgrid(np.arange(25, 30), np.arange(0, 25))
x_coords, y_coords = raster_tile.rc2xy(rows.flatten(), cols.flatten())
x_coords, y_coords

In [ ]:
rows, cols = raster_tile.xy2rc(x_coords, y_coords)
rows, cols

### Magic Methods

For some of the aforementioned functions a set of magic methods are available to enable a Pythonic usage of a raster tile. This involves for instance the WITHIN check, which is internally called when using Pythons `in`.

In [ ]:
raster_tile_within in raster_tile, raster_tile_outer in raster_tile

Two raster tiles are considered (spatially) the same if they have the same outer boundary corners, number of rows and number of cols.

In [ ]:
raster_tile_eq = RasterTile(
    n_rows=n_rows, n_cols=n_cols, crs=epsg, geotrans=geotrans, name="Tile 2"
)

raster_tile == raster_tile_eq, raster_tile == raster_tile_overlap

The `str` method returns the WKT representation of the raster tile.

In [ ]:
str(raster_tile)